<a href="https://colab.research.google.com/github/kbrig573/MinderTesting/blob/Develop/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-groq langchain-community langchain-huggingface chromadb pypdf sentence-transformers

In [ ]:
"""
RAG (Retrieval Augmented Generation) Chatbot
-------------------------------------------
This implementation follows the RAG architecture with clear separation of components:
1. Client Interface: Handles user interaction
2. Vector Database: Stores and retrieves document embeddings
3. Framework: Manages the RAG pipeline
4. LLM Integration: Handles the language model interaction
"""

import os
from typing import Dict, List, Tuple
from dataclasses import dataclass
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
# Configuration
@dataclass
class RAGConfig:
    """Configuration settings for the RAG system"""
    chunk_size: int = 500
    chunk_overlap: int = 150
    model_name: str = "mixtral-8x7b-32768"
    temperature: float = 0.7
    max_tokens: int = 4096
    top_k_results: int = 4

In [ ]:
class DocumentProcessor:
    """Component 1: Handles document loading and preprocessing"""
    def __init__(self, config: RAGConfig):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap,
            separators=["\n\n", "\n", ".", " "]
        )

    def load_and_split_document(self, pdf_path: str) -> List:
        """Load PDF and split into chunks"""
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        return self.text_splitter.split_documents(documents)

In [ ]:
class VectorStore:
    """Component 2: Handles vector database operations"""
    def __init__(self, pdf_path: str):
        self.persist_dir = f"chroma_db_{pdf_path}"
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cuda'}
        )

    def initialize_store(self, documents: List) -> Chroma:
        """Initialize or load vector store"""
        if os.path.exists(self.persist_dir):
            print("Loading existing vector store...")
            return Chroma(
                persist_directory=self.persist_dir,
                embedding_function=self.embeddings
            )

        print("Creating new vector store...")
        db = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=self.persist_dir
        )
        return db

In [ ]:

class PromptManager:
    """Component 3: Handles prompt engineering and context formatting"""
    @staticmethod
    def create_retrieval_prompt(question: str) -> str:
        return f"""Please analyze the provided context carefully and answer the following question.
        If the answer isn't clearly supported by the context, please indicate that.

        Question: {question}
        """

    @staticmethod
    def format_sources(sources: List) -> str:
        formatted_sources = []
        for idx, doc in enumerate(sources, 1):
            page_info = f"(Page {doc.metadata.get('page', 'N/A')})"
            source_text = f"{doc.page_content[:200]}..."
            formatted_sources.append(f"{idx}. {source_text} {page_info}")
        return "\n\n".join(formatted_sources)

In [ ]:
class RAGChatbot:
    """Main RAG Framework: Orchestrates the entire RAG process"""
    def __init__(self, pdf_path: str, config: RAGConfig = RAGConfig()):
        self.config = config

        # Initialize components
        print("1. Initializing document processor...")
        self.doc_processor = DocumentProcessor(config)

        print("2. Processing document...")
        documents = self.doc_processor.load_and_split_document(pdf_path)

        print("3. Setting up vector store...")
        self.vector_store = VectorStore(pdf_path)
        self.vectordb = self.vector_store.initialize_store(documents)

        print("4. Setting up LLM and conversation chain...")
        self.setup_llm_chain()

        self.prompt_manager = PromptManager()

    def setup_llm_chain(self):
        """Set up the LLM and conversation chain"""
        llm = ChatGroq(
            model_name=self.config.model_name,
            temperature=self.config.temperature,
            max_tokens=self.config.max_tokens
        )

        memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            return_messages=True
        )

        self.chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=self.vectordb.as_retriever(
                search_type="mmr",
                search_kwargs={
                    "k": self.config.top_k_results,
                    "fetch_k": self.config.top_k_results * 2,
                    "lambda_mult": 0.7
                }
            ),
            memory=memory,
            return_source_documents=True,
            verbose=False
        )

    def chat(self, question: str) -> Dict:
        """Process a question through the RAG pipeline"""
        try:
            # 1. Format the question with proper prompt
            formatted_question = self.prompt_manager.create_retrieval_prompt(question)

            # 2. Get response from LLM with retrieved context
            result = self.chain.invoke({"question": formatted_question})

            # 3. Format the response and sources
            return {
                "answer": result["answer"],
                "sources": self.prompt_manager.format_sources(result["source_documents"])
            }
        except Exception as e:
            print(f"Error in RAG pipeline: {str(e)}")
            return {
                "answer": "An error occurred while processing your question.",
                "sources": []
            }



In [ ]:
def main():
    """Client Interface: Handles user interaction"""
    # Initialize with configuration
    config = RAGConfig(
        chunk_size=500,
        chunk_overlap=150,
        temperature=0.7,
        top_k_results=4
    )

    # Create chatbot instance
    pdf_path = "/content/1706.03762v7.pdf"  # Attention is All You Need paper
    chatbot = RAGChatbot(pdf_path, config)

    print("\nRAG Chatbot initialized! Type 'exit' to end the conversation.")
    print("This chatbot uses the following components:")
    print("1. Document Processing: Splits PDF into manageable chunks")
    print("2. Vector Database: Stores and retrieves relevant context")
    print("3. LLM Integration: Generates responses using Groq's Mixtral model")
    print("4. RAG Framework: Orchestrates the entire process\n")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            break

        response = chatbot.chat(user_input)
        print("\nChatbot:", response["answer"])
        if response["sources"]:
            pass
            # print("\nSources used:")
            # print(response["sources"][0])

if __name__ == "__main__":
    main()

In [ ]:
!pip install -q groq

In [ ]:
# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llava-v1.5-7b-4096-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {"type": "image_url", "image_url": {"url": "https://media.istockphoto.com/id/961805314/photo/a-new-beginning-into-a-sunny-future.jpg?s=1024x1024&w=is&k=20&c=Msk83herhD09mea1_XahyeL7yWMHkLZ4_Eq8laT59qM="}}
            ]
        },
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

print(completion.choices[0].message.content)